In [25]:
# this program will call maps.nyc to get the precinct number for each high school

# dependencies
import requests
import pandas as pd
import numpy as np

# files to load
hs_directory = "Resources/2018_DOE_High_School_Directory.csv"

# read file and store into pandas data frame
high_schools = pd.read_csv(hs_directory)

In [26]:
# select required columns
high_schools = high_schools[["dbn", "primary_address_line_1", "Borough"]]

In [27]:
# clean data
high_schools["primary_address_line_1"] = high_schools["primary_address_line_1"].replace('- ', '-', regex=True)
high_schools["Borough"][391] = "QUEENS"
high_schools["Borough"] = high_schools["Borough"].replace('STATEN IS', 'STATEN ISLAND', regex=True)

# replace spaces with %20 prior to creating query url
high_schools["primary_address_line_1"] = high_schools["primary_address_line_1"].replace(' ', '%20', regex=True)

# add column to store precinct numbers
high_schools["Precinct"] = np.nan

In [28]:
# call api for each hs address and store precinct number
i = 0
for row in high_schools["primary_address_line_1"]:
    address = high_schools["primary_address_line_1"][i]
    borough = high_schools["Borough"][i].strip()
    url = f"https://maps.nyc.gov/geoclient/v1/search.json?app_key=E2857975AA57366BC&app_id=nyc-gov-nypd&input={address},%20{borough}"
    query = requests.get(url).json()
    high_schools["Precinct"][i] = query["results"][0]["response"]["policePrecinct"]
    i += 1

C:\Users\Allen\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [29]:
# remove unneeded columns and create excel file
high_schools = high_schools.drop(columns=["primary_address_line_1", "Borough"])
high_schools.to_excel("Resources/Precinct_Key.xlsx", index=False)